In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from google.colab import drive, files
import io

# 5. Stop words, lematizing, stemming

## Cargar datos y dividir dataset

In [ ]:
from tensorflow.random import set_seed
set_seed(234730)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_without_label.csv')

In [ ]:
train['id'] = train['id'].astype(str)
train['tweet'] = train['tweet'].astype(str)
train['label'] = train['label'].astype(str)

In [ ]:
train.drop(['id'], axis=1)
train.set_index("id", inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(
    train.drop('label', axis=1),
    train['label'],
    test_size=(1.0/3), random_state=42)

## Stop words y lemmatize

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')
Lemmatizer = WordNetLemmatizer()
#analyzer = CountVectorizer().build_analyzer()

from zipfile import ZipFile
file_loc = '/root/nltk_data/corpora/wordnet.zip'
with ZipFile(file_loc, 'r') as z:
  z.extractall('/root/nltk_data/corpora/')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#Lemmatizer
def lemmatized_words(doc):
  return (Lemmatizer.lemmatize(w.lower()) for w in doc.split())

#CountVectorizer con stopwords de Natural Language Toolkit
nltk.download('stopwords')

nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

## CountVectorizer con los nuevos parametros

In [ ]:
vectorizer_stop_words_stemmed = CountVectorizer(stop_words = nltk.corpus.stopwords.words('english'), analyzer=lemmatized_words)
bag_of_words = vectorizer_stop_words_stemmed.fit(X_train["tweet"])

print(bag_of_words.get_feature_names())

Hay muchas palabras que estan repetidas como "august", "august,", "august.", "august:", etc. Hay que eliminar esas palabras y dejarlas como una sola. También se nota que hay muchos emojis y signos, que habría que ver como los vamos a manejar

In [ ]:
bag_of_words = vectorizer_stop_words_stemmed.transform(X_train["tweet"])
bag_of_words = pd.DataFrame(bag_of_words.toarray(), columns = vectorizer_stop_words_stemmed.get_feature_names())

In [ ]:
bag_of_words.shape

(5706, 23976)

In [ ]:
model = Sequential()
model.add(Dense(8, input_shape=(13,)))
model.add(Activation('relu'))
model.add(Dense(1))